In [51]:
import rclpy
print("ROS 2 is available!")

ROS 2 is available!


In [1]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import Bool
from geometry_msgs.msg import Twist
from rclpy.qos import QoSProfile, QoSReliabilityPolicy, QoSDurabilityPolicy
import math
import time
from px4_msgs.msg import TrajectorySetpoint
from geometry_msgs.msg import Twist, Point
from geometry_msgs.msg import PointStamped

class DroneCommander(Node):
    def __init__(self):
        super().__init__('drone_commander')

        qos_profile = QoSProfile(
            reliability=QoSReliabilityPolicy.BEST_EFFORT,
            durability=QoSDurabilityPolicy.TRANSIENT_LOCAL,
            depth=1
        )

        self.arm_pub = self.create_publisher(Bool, '/arm_message', qos_profile)
        self.position_pub = self.create_publisher(TrajectorySetpoint, '/position_goal', qos_profile)
        self.vel_pub = self.create_publisher(Twist, '/offboard_velocity_cmd', qos_profile)
        self.publisher = self.create_publisher(PointStamped, '/goal_point', qos_profile)

    def publish_arm(self, value: bool):
        msg = Bool()
        msg.data = value
        self.arm_pub.publish(msg)
        self.get_logger().info(f"Sent ARM: {value}")

    def publish_position_goal(self, x, y, z, yaw=None):
        msg = TrajectorySetpoint()
        msg.position = [x, y, z]
        msg.velocity = [float('nan')] * 3
        msg.acceleration = [float('nan')] * 3
        msg.yaw = float('nan') if yaw is None else yaw
        msg.yawspeed = 0.0
        self.position_pub.publish(msg)
        self.get_logger().info(f"Published Position Goal: {[x, y, z]}")
    
    def publish_velocity(self, vx: float, vy: float, vz: float, yaw_rate: float = 0.0):
        twist = Twist()
        twist.linear.x = vx
        twist.linear.y = vy
        twist.linear.z = vz
        twist.angular.z = yaw_rate
        self.vel_pub.publish(twist)
        
    def send_goal(self, x, y, z=1.0, frame='map'):
        msg = PointStamped()
        msg.header.frame_id = frame
        msg.header.stamp = self.get_clock().now().to_msg()
        msg.point.x = x
        msg.point.y = y
        msg.point.z = z
        self.publisher.publish(msg)
        print(f"Sent goal: ({x}, {y}, {z}) in frame '{frame}'")



In [2]:
# Shutdown any old instance
try:
    rclpy.shutdown()
except:
    pass

# Init ROS
rclpy.init()
node = DroneCommander()
rclpy.spin_once(node, timeout_sec=0.1)

In [83]:
### Arm
node.publish_arm(True)
rclpy.spin_once(node, timeout_sec=0.1)

# Wait for drone to arm and takeoff sequence to finish
time.sleep(10)

[INFO] [1750150653.901802423] [drone_commander]: Sent ARM: True


In [101]:
### Circle flight
radius = 1.0  # meters/sec linear speed
duration = 10.0  # seconds
hz = 20
steps = int(duration * hz)

for i in range(steps):
    t = i / hz
    vx = radius * math.cos(2 * math.pi * t / duration)
    vy = radius * math.sin(2 * math.pi * t / duration)
    vz = 0.0
    yaw_rate = 0.0
    node.publish_velocity(vx, vy, vz, yaw_rate)
    rclpy.spin_once(node, timeout_sec=0.01)
    time.sleep(1 / hz)

# Stop motion
node.publish_velocity(0.0, 0.0, 0.0, 0.0)
time.sleep(1)

In [99]:
### Send a position goal
x = 0.0
y = 1.0
z = 4.0
node.publish_position_goal(x=y, y=x, z=-z)  # 3D position
# Wait to reach goal
time.sleep(10)

[INFO] [1750152304.860142543] [drone_commander]: Published Position Goal: [1.0, 0.0, -4.0]


In [ ]:
#sudo apt install ros-${ROS_DISTRO}-geographic-msgs
#ros2 topic echo /fmu/out/vehicle_global_position (to get current GPS->STARTING POSTION OF DRONE IN SIMULATION)
# new_lat = 47.3979715 + 0.00001  # ~1.11 meters north
# new_lon = 8.5461636 + 0.00001   # ~0.74 meters east
# new_alt = 2.0                   # 2 meters altitude


In [ ]:
# # Publish GPS waypoint
# node.publish_gps_waypoint(47.3979815, 8.5461736, 2.0)
# time.sleep(10)

In [5]:
#Navigation Goal-> "Direct Path Navigation" or "Quasi-Navigation"
node.send_goal(2.0,-2.0,0.0)
rclpy.spin_once(node, timeout_sec=0.01)

Sent goal: (2.0, -2.0, 0.0) in frame 'map'


In [ ]:
### Disarm --> NOT WORKING
# node.publish_arm(False)
# rclpy.spin_once(node, timeout_sec=0.1)

In [11]:
# Cleanup
node.destroy_node()
rclpy.shutdown()